In [25]:
import numpy as np
import matplotlib.pyplot as plt

def flujo_carga_newton_raphson(n_barras, lineas, datos_barras, tol=1e-8, max_iter=300):
    # Datos de las barras
    tipo = datos_barras[:, 0]  # Tipo de barra (1=slack, 2=PQ (1A), 3=PQ (2A), 4=PQ (3A), 5=PQ (2B), 6=PQ (1B) )
    Pesp = datos_barras[:, 1]  # Potencia activa especificada (MW)
    Qesp = datos_barras[:, 2]  # Potencia reactiva especificada (Mvar)
    Vesp = datos_barras[:, 3]  # Voltaje especificado (p.u.)
    ang_esp = datos_barras[:, 4]  # Ángulo especificado (rad)

import numpy as np
import matplotlib.pyplot as plt

def flujo_carga_newton_raphson(n_barras, lineas, datos_barras, tol=1e-8, max_iter=300):
    # Datos de las barras
    tipo = datos_barras[:, 0]  # Tipo de barra (1=slack, 2=PQ (1A), 3=PQ (2A), 4=PQ (3A), 5=PQ (2B), 6=PQ (1B) )
    Pesp = datos_barras[:, 1]  # Potencia activa especificada (MW)
    Qesp = datos_barras[:, 2]  # Potencia reactiva especificada (Mvar)
    Vesp = datos_barras[:, 3]  # Voltaje especificado (p.u.)
    ang_esp = datos_barras[:, 4]  # Ángulo especificado (rad)

    # Inicialización de tensiones y ángulos
    V = Vesp.copy()
    ang = ang_esp.copy()

    # Matriz de admitancia
    Y = np.zeros((n_barras, n_barras), dtype=complex)
    for linea in lineas:
        de, a, r, x, b = linea
        Y[int(de)-1, int(a)-1] = -1 / (r + 1j * x)
        Y[int(a)-1, int(de)-1] = Y[int(de)-1, int(a)-1]
    for i in range(n_barras):
        Y[i, i] = -np.sum(Y[i, :]) + 1j * datos_barras[i, 4] / 2

    # Imprimir la matriz de admitancia (Ybus)
    print("Matriz de admitancia (Ybus):")
    print(Y)

    # Iteración de Newton-Raphson
    for iter in range(max_iter):
        # Cálculo de P y Q calculados
        P_calc = np.zeros(n_barras)
        Q_calc = np.zeros(n_barras)
        for i in range(n_barras):
            for k in range(n_barras):
                P_calc[i] += V[i] * V[k] * (Y[i, k].real * np.cos(ang[i] - ang[k]) + Y[i, k].imag * np.sin(ang[i] - ang[k]))
                Q_calc[i] += V[i] * V[k] * (Y[i, k].real * np.sin(ang[i] - ang[k]) - Y[i, k].imag * np.cos(ang[i] - ang[k]))

        # Vector de diferencias
        dP = Pesp - P_calc
        dQ = Qesp - Q_calc

        # Construcción de la matriz Jacobiana
        J = np.zeros((2 * (n_barras - 1), 2 * (n_barras - 1)))

        # Submatriz J1 (Derivadas de P respecto a ángulos)
        for i in range(n_barras - 1):
            for k in range(n_barras - 1):
                if i == k:
                    J[i, k] = -Q_calc[i] - V[i] ** 2 * Y[i, i].imag
                else:
                    J[i, k] = V[i] * V[k] * (Y[i, k].real * np.sin(ang[i] - ang[k]) - Y[i, k].imag * np.cos(ang[i] - ang[k]))

        # Submatriz J2 (Derivadas de P respecto a V)
        for i in range(n_barras - 1):
            for k in range(n_barras - 1):
                if i == k:
                    J[i, k + (n_barras - 1)] = P_calc[i] / V[i] + V[i] * Y[i, i].real
                else:
                    J[i, k + (n_barras - 1)] = V[i] * (Y[i, k].real * np.cos(ang[i] - ang[k]) + Y[i, k].imag * np.sin(ang[i] - ang[k]))

        # Submatriz J3 (Derivadas de Q respecto a ángulos)
        for i in range(n_barras - 1):
            for k in range(n_barras - 1):
                if i == k:
                    J[i + (n_barras - 1), k] = P_calc[i] - V[i] ** 2 * Y[i, i].real
                else:
                    J[i + (n_barras - 1), k] = V[i] * V[k] * (-Y[i, k].real * np.cos(ang[i] - ang[k]) - Y[i, k].imag * np.sin(ang[i] - ang[k]))

        # Submatriz J4 (Derivadas de Q respecto a V)
        for i in range(n_barras - 1):
            for k in range(n_barras - 1):
                if i == k:
                    J[i + (n_barras - 1), k + (n_barras - 1)] = Q_calc[i] / V[i] - V[i] * Y[i, i].imag
                else:
                    J[i + (n_barras - 1), k + (n_barras - 1)] = V[i] * (-Y[i, k].real * np.sin(ang[i] - ang[k]) + Y[i, k].imag * np.cos(ang[i] - ang[k]))

        # Vector de diferencias (dP y dQ)
        F = np.concatenate([dP[:n_barras - 1], dQ[:n_barras - 1]])

        # Resolver el sistema de ecuaciones lineales J * delta = -F
        delta = np.linalg.solve(J, -F)

        # Actualizar ángulos y tensiones
        ang[:n_barras - 1] += delta[:n_barras - 1]
        V[:n_barras - 1] += delta[n_barras - 1:]

        # Comprobar la convergencia
        if np.max(np.abs(F)) < tol:
            print(f"Convergencia alcanzada en {iter + 1} iteraciones.")
            return V, ang, iter + 1

    print("No se alcanzó la convergencia.")
    return V, ang, iter + 1

# Definir las líneas y las barras
lineas = [
    # De, A, R, X, B (en pu)
    [1, 2, 0.02, 0.06, 0.03],
    [2, 3, 0.05, 0.19, 0.02],
    [3, 4, 0.05, 0.19, 0.02],
    [4, 5, 0.03, 0.17, 0.01],
    [5, 6, 0.04, 0.17, 0.02],
    [6, 1, 0.02, 0.06, 0.03]  # Conexión de la barra 6 de vuelta a la barra 1
]

# Definir los datos de las barras
S_base = 100  # Potencia base en MVA
P_base = 150  # Potencia activa base en MW
Q_base = 100  # Potencia reactiva base en Mvar

datos_barras = np.array([
    [1, 0.0, 0.0, 1.0, 0.0],               # Barra 2 (Slack) V = 1 p.u., ángulo = 0 grados
    [2, -0.6 * P_base, -0.6 * Q_base, 1.0, 0.0],  # Barra 3 (PQ)
    [3, -0.35 * P_base, -0.35 * Q_base, 1.0, 0.0],  # Barra 4 (PQ)
    [4, -0.15 * P_base, -0.15 * Q_base, 1.0, 0.0],  # Barra 5 (PQ)
    [5, -0.6 * P_base, -0.6 * Q_base, 1.0, 0.0],   # Barra 6 (PQ)
    [6, -0.1 * P_base, -0.1 * Q_base, 1.0, 0.0]    # Barra 7 (PQ)
])

n_barras = len(datos_barras)

# Resolver flujo de carga
V, ang, iteraciones = flujo_carga_newton_raphson(n_barras, lineas, datos_barras)

# Mostrar resultados
for i in range(n_barras):
    print(f"Barra {i + 1}: V = {V[i]:.4f} p.u., Ángulo = {ang[i] * 180 / np.pi:.2f}°")

print(f"\nNúmero de iteraciones: {iteraciones}")


Matriz de admitancia (Ybus):
[[10.        -30.j         -5.        +15.j
   0.         +0.j          0.         +0.j
   0.         +0.j         -5.        +15.j        ]
 [-5.        +15.j          6.29533679-19.92227979j
  -1.29533679 +4.92227979j  0.         +0.j
   0.         +0.j          0.         +0.j        ]
 [ 0.         +0.j         -1.29533679 +4.92227979j
   2.59067358 -9.84455959j -1.29533679 +4.92227979j
   0.         +0.j          0.         +0.j        ]
 [ 0.         +0.j          0.         +0.j
  -1.29533679 +4.92227979j  2.3020482 -10.62697778j
  -1.00671141 +5.70469799j  0.         +0.j        ]
 [ 0.         +0.j          0.         +0.j
   0.         +0.j         -1.00671141 +5.70469799j
   2.31818682-11.27846848j -1.31147541 +5.57377049j]
 [-5.        +15.j          0.         +0.j
   0.         +0.j          0.         +0.j
  -1.31147541 +5.57377049j  6.31147541-20.57377049j]]
No se alcanzó la convergencia.
Barra 1: V = -165026094106564948353694310297904933965

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandapower as pp
import math

# Definimos los parámetros
f = 50  # frecuencia del sistema en [Hz]

Vg_LL = 110  # voltaje de gen [kV]
Vg_LL2 = 220 # Voltaje de barras
Imax = 0.457  # corriente máxima en las líneas [kA]

# Respecto a la carga
P = 150  # potencia activa en [MW]
Q = 100  # potencia reactiva en [Mvar]

# Crear red eléctrica
net = pp.create_empty_network()

# Añadir barras
bus1 = pp.create_bus(net, vn_kv=Vg_LL, name="Barra 1", slack=True)  # Barra 1 es designada como slack
bus2 = pp.create_bus(net, vn_kv=Vg_LL2, name="Barra 2")  # Barra 2 se conecta la carga
bus1A = pp.create_bus(net, vn_kv=Vg_LL2, name="Barra 1A")  # Barra 1A se conecta la carga
bus2A = pp.create_bus(net, vn_kv=Vg_LL2, name="Barra 2A")  # Barra 2A se conecta la carga
bus3A = pp.create_bus(net, vn_kv=Vg_LL2, name="Barra 3A")  # Barra 3A se conecta la carga
bus2B = pp.create_bus(net, vn_kv=Vg_LL2, name="Barra 2B")  # Barra 2B se conecta la carga
bus1B = pp.create_bus(net, vn_kv=Vg_LL2, name="Barra 1B")  # Barra 1B se conecta la carga

# Añadir líneas
pp.create_line(net, from_bus=bus2, to_bus=bus1A, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", in_service=True)
pp.create_line(net, from_bus=bus1A, to_bus=bus2A, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", in_service=True)
pp.create_line(net, from_bus=bus2A, to_bus=bus3A, length_km=20, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", in_service=True)
pp.create_line(net, from_bus=bus3A, to_bus=bus2B, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", in_service=True)
pp.create_line(net, from_bus=bus2B, to_bus=bus1B, length_km=30, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", in_service=True)
pp.create_line(net, from_bus=bus1B, to_bus=bus2, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", in_service=True)

# Añadir carga
pp.create_load(net, bus=bus1A, p_mw=0.2*P, q_mvar=0.2*Q)
pp.create_load(net, bus=bus2A, p_mw=0.35*P, q_mvar=0.35*Q)
pp.create_load(net, bus=bus3A, p_mw=0.15*P, q_mvar=0.15*Q)
pp.create_load(net, bus=bus2B, p_mw=0.6*P, q_mvar=0.6*Q)
pp.create_load(net, bus=bus1B, p_mw=0.1*P, q_mvar=0.1*Q)

# Se añade transformador de 100 MVA 220/110 kV entre la barra 1 y la barra 2
pp.create_transformer(net, hv_bus=bus2, lv_bus=bus1, name="trafo1", std_type="100 MVA 220/110 kV")

# Se añade fuente de generación conectada en la barra 1, (1[pu], el voltaje nominal [kV] se define en la barra a la que está conectada)
pp.create_ext_grid(net, bus=bus1, vm_pu=1, va_degree=0.0)

# Contador de iteraciones
iteraciones = 0

# Ejecutar el flujo de carga
success = False
while not success:
    iteraciones += 1
    pp.runpp(net, tolerance_mva=1e-8, max_iteration=1000)  # No es necesario ajustar max_iteration
    
    # Verificar si el flujo de carga convergió
    if net.converged:
        success = True

# Obtener resultados de las tensiones y ángulos en las barras
voltajes = net.res_bus.vm_pu.values  # Voltajes en p.u.
angulos = net.res_bus.va_degree.values  # Ángulos en grados

# Imprimir los resultados
for i in range(len(net.bus)):
    nombre = net.bus.name[i]
    voltaje = voltajes[i]
    angulo = angulos[i]
    print(f'Barra {nombre}: Voltaje = {voltaje:.4f} p.u., Ángulo = {angulo:.2f} grados')

print(f'\nNúmero de iteraciones: {iteraciones}')
print(f'Tolerancia utilizada: {1e-8:.2e} MVA')  # Mostramos la tolerancia utilizada, que es la especificada






/Users/josegutierrez/Library/Python/3.9/lib/python/site-packages/pandapower/create.py:2369: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  net.trafo.tap_phase_shifter.fillna(False, inplace=True)
numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)



Barra Barra 1: Voltaje = 1.0000 p.u., Ángulo = 0.00 grados
Barra Barra 2: Voltaje = 1.0334 p.u., Ángulo = -14.28 grados
Barra Barra 1A: Voltaje = 1.0315 p.u., Ángulo = -14.49 grados
Barra Barra 2A: Voltaje = 1.0292 p.u., Ángulo = -14.73 grados
Barra Barra 3A: Voltaje = 1.0284 p.u., Ángulo = -14.86 grados
Barra Barra 2B: Voltaje = 1.0274 p.u., Ángulo = -14.87 grados
Barra Barra 1B: Voltaje = 1.0324 p.u., Ángulo = -14.47 grados

Número de iteraciones: 1
Tolerancia utilizada: 1.00e-08 MVA
